In [5]:
import secrets, base64

Step 1: Make Secret


In [12]:
def makeSecret():
    secret = secrets.token_bytes(20)
    base32 = base64.b32encode(secret).decode("utf-8").replace("=","")
    return base32
makeSecret()

'XSG7YLAYSP5GEYFEDETJ4LXMIKFGIOH3'